In [25]:
import pandas as pd
import os
from laserembeddings import Laser
import numpy as np
# import preprocessor 
# import nltk
# nltk.download('punkt')
# from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
# import fasttext
# import csv
# import datetime
# from bs4 import BeautifulSoup
# import re
# import itertools
# import emoji
# from xgboost import XGBClassifier
# from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC

In [2]:
folder_path = "../Data Twitter/Train/"
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('txt'):
        path = os.path.join(folder_path,filename)
        df = pd.read_csv(path,sep='\t',header=None)
        df = df.drop(columns=[df.columns[0],df.columns[3]])
        dfs.append(df)

df_train = pd.concat(dfs)
df_train.columns = ['text','label']
df_train['label_numeric'] = df_train['label'].astype('category').cat.codes


folder_path = "../Data Twitter/Test/"
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('txt'):
        path = os.path.join(folder_path,filename)
        df = pd.read_csv(path,sep='\t',header=None)
        df = df.drop(columns=[df.columns[0],df.columns[3]])
        dfs.append(df)

df_train_2 = pd.concat(dfs)
df_train_2.columns = ['text','label']
df_train_2['label_numeric'] = df_train_2['label'].astype('category').cat.codes

df_train = df_train.append(df_train_2).reset_index()
print('total train samples : ',len(df_train))
print(df_train['label'].value_counts())

class_mapping = list(df_train['label'].astype('category').cat.categories)

total train samples :  6755
fear       2142
anger      1617
joy        1537
sadness    1459
Name: label, dtype: int64


In [3]:
folder_path = "../Data Twitter/Dev/"
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('txt'):
        path = os.path.join(folder_path,filename)
        df = pd.read_csv(path,sep='\t',header=None)
        df = df.drop(columns=[df.columns[0],df.columns[3]])
        dfs.append(df)

df_dev = pd.concat(dfs)
df_dev.columns = ['text','label']
df_dev['label_numeric'] = df_dev['label'].astype('category').cat.codes

folder_path = "../Data Twitter/Dev/without intensity"
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('txt'):
        path = os.path.join(folder_path,filename)
        df = pd.read_csv(path,sep='\t',header=None)
        df = df.drop(columns=[df.columns[0],df.columns[3]])
        dfs.append(df)

df_dev_2 = pd.concat(dfs)
df_dev_2.columns = ['text','label']
df_dev_2['label_numeric'] = df_dev_2['label'].astype('category').cat.codes

df_dev = df_dev.append(df_dev_2).reset_index()
print('total train samples : ',len(df_dev))
print(df_dev['label'].value_counts())

total train samples :  694
fear       220
anger      168
joy        158
sadness    148
Name: label, dtype: int64


## Prepare Embeddings

In [5]:
laser = Laser()

In [23]:
X_train = laser.embed_sentences(df_train['text'].tolist(),'en')
X_test = laser.embed_sentences(df_dev['text'].tolist(),'en')

In [26]:
Y_train = df_train['label'].tolist()
Y_test = df_dev['label'].tolist()

## Train Model

In [29]:
model = LogisticRegression(max_iter=200)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=200)

In [30]:
Y_pred = model.predict(X_test)
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

       anger       0.64      0.63      0.63       168
        fear       0.64      0.73      0.68       220
         joy       0.66      0.70      0.68       158
     sadness       0.64      0.49      0.55       148

    accuracy                           0.65       694
   macro avg       0.65      0.64      0.64       694
weighted avg       0.65      0.65      0.64       694

